# 제 8 장 __인공신경망의 이해와 활용__
___

## __사전설정__
---

(1) 저장소 데이터 가져오기

In [ ]:
!rm -rf /content/BizDataAnalysis/

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git


(2) matplotlib 환경에서 한글 사용

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

런타임 메뉴에서 '세션 다시 시작' 후 다음의 명령문 실행

In [ ]:
from matplotlib import rc
rc('font', family='NanumGothicCoding')
rc('axes', unicode_minus=False)

(3) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

___

## 8.2 __인공신경망의 이해__

### (1) 예측모형을 인공신경망으로 구현한 사례

* Boston house-price 데이터 가져오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
boston = pd.read_table('/content/BizDataAnalysis/DATA/Boston.txt', sep='\s+')

In [ ]:
boston.head()

__1.  학습데이터와 평가데이터 데이터분할__

* 강의노트에 있는 scikit-learn 라이브러리에서 제공하는 기능을 사용해도 동일한 결과를 얻을 수 있음

In [ ]:
X_train = boston.sample(frac=0.7,random_state=3)
X_test = boston.drop(X_train.index)

In [ ]:
y_train = X_train.pop('MEDV')
y_test = X_test.pop('MEDV')

In [ ]:
#@title 데이터 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

__2.  분석에 사용할 신경망 모형 설정__

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
boston_nn_mod = Sequential(name='Boston_NN')

In [ ]:
boston_nn_mod.add(Dense(2,input_dim=13,activation=None,name="Hidden_Layer_1"))

In [ ]:
#@title 활성함수 변경 사례
boston_nn_mod.add(Dense(2,input_dim=13,activation='relu',name="Hidden_Layer_1"))

In [ ]:
#@title 은닉층 추가 사례
boston_nn_mod.add(Dense(2,input_dim=2,activation='relu',name="Hidden_Layer_2"))

In [ ]:
boston_nn_mod.add(Dense(1,name="Output_layer"))

In [ ]:
boston_nn_mod.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(boston_nn_mod, show_shapes=True)

In [ ]:
ex_batch = X_train[:10]
ex_result = boston_nn_mod.predict(ex_batch)
ex_result

__3.  모형 컴파일__

In [ ]:
boston_nn_mod.compile(optimizer='rmsprop', loss='mse', metrics=['mae','mse'])

__4.  모형의 학습과 평가__

In [ ]:
history = boston_nn_mod.fit(
  X_train, y_train,
  validation_data=(X_test,y_test),
  # epochs=300
  epochs=10
)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error')
plt.plot(hist['epoch'], hist['mae'],label='Train Error')
plt.plot(hist['epoch'], hist['val_mae'],label = 'Test Error')
#  plt.ylim([0,5])
plt.legend()
plt.show()


In [ ]:
plt.xlabel('Epoch')
plt.ylabel('Mean Square Error')
plt.plot(hist['epoch'], hist['mse'],label='Train Error')
plt.plot(hist['epoch'], hist['val_mse'],label = 'Test Error')
#plt.ylim([0,20])
plt.legend()
plt.show()

In [ ]:
score = boston_nn_mod.evaluate(X_test, y_test)
print(f'Test Loss : {score[0]}')
print(f'Test Mae  : {score[1]}')

__5.  모형의 예측결과 살펴보기__

In [ ]:
y_test_pred = boston_nn_mod.predict(X_test)

In [ ]:
boston_nn_mod.predict(X_test)

In [ ]:
xref = np.arange(0,60,0.1)
yref = xref
plt.plot(xref,yref,c='red')
plt.scatter(y_test,np.array(y_test_pred).T[0])
plt.title('평가데이터에서 실제값 vs. 예측값 ')
plt.xlabel('실제값')
plt.ylabel('예측값')
plt.show()

__6.  모형의 저장__

In [ ]:
boston_nn_mod.save("Boston prediction NN model") # 모형 저장

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("Boston prediction NN model")

In [ ]:
model.summary()

---

### (2) 분류모형을 인공신경망으로 구현한 사례

* 연체 데이터 가져오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
default = pd.read_table('/content/BizDataAnalysis/DATA/Default.csv', sep=',')

In [ ]:
default.head()

* 독립변수와 종속변수로 분리 및 가변수 처리

In [ ]:
X = default.iloc[:,2:]
y = default.iloc[:,[1]]
X = pd.get_dummies(X,drop_first=True,dtype=int)
y = pd.get_dummies(y,drop_first=True,dtype=int)

__1.  학습데이터와 평가데이터 데이터분할__

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=3,train_size=0.7)

In [ ]:
#@title 데이터 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#@title 데이터를 0과 1사이의 값으로 변환
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

__2.  분석에 사용할 신경망 모형 설정__

In [ ]:
default_nn_mod = Sequential(name='Default_NN')

In [ ]:
default_nn_mod.add((Dense(64,input_dim=3,activation='relu',name="Hidden_Layer_1")))
default_nn_mod.add((Dense(64,activation='relu',name="Hidden_Layer_2")))
default_nn_mod.add((Dense(1,activation='sigmoid',name="Output_Layer")))

In [ ]:
default_nn_mod.summary()

__3.  모형 컴파일__

In [ ]:
default_nn_mod.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

__4.  모형의 학습과 평가__

In [ ]:
history = default_nn_mod.fit(
  X_train, y_train,
  validation_split = 0.2,
  epochs=500,
  batch_size=64,
  verbose=0
)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(hist['epoch'], hist['acc'],label='Train Error')
plt.plot(hist['epoch'], hist['val_acc'],label = 'Val Error')
plt.legend()
plt.show()

__5.  모형의 예측결과 살펴보기__

In [ ]:
y_test_prob = default_nn_mod.predict(X_test)

In [ ]:
y_test_prob > 0,5

In [ ]:
y_test_pred = np.where(y_test_prob > 0.5 , 1, 0 )

In [ ]:
pd.crosstab(index=np.array(y_test['default_Yes']),columns=np.concatenate(y_test_pred))